In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.metrics import r2_score, mean_absolute_error
from gpytorch.mlls import ExactMarginalLogLikelihood
import gpytorch
import altair as alt
import matplotlib.pyplot as plt
import re
from scipy.stats import norm
import os

UTILITY

In [2]:
import numpy as np
import matplotlib.pyplot as plt

def plot_gp_mean_with_uncertainty(candidates, mean, var, iteration, label="target_value", save=False, fname_prefix="gp_mean"):
    """
    Plottet pro Iteration die GP-Vorhersage (Mean) über alle Kandidaten-Strukturen
    inkl. Unsicherheitsband (±1σ).

    Args:
        candidates (pd.DataFrame): candidates, in exakt der Reihenfolge wie test_x berechnet wurde
        mean (torch.Tensor): prediction.mean (shape [N])
        var (torch.Tensor): prediction.variance (shape [N])
        iteration (int): Iterationsnummer für Titel/Dateiname
        label (str): Labelname (nur für Titel)
        save (bool): falls True, speichert PNG
        fname_prefix (str): Dateiprefix falls save=True
    """
    # --- to numpy ---
    mean_np = mean.detach().cpu().numpy().flatten()
    var_np  = var.detach().cpu().numpy().flatten()
    std_np  = np.sqrt(var_np)
    std_np  = np.maximum(std_np, 1e-12)  # Schutz gegen 0

    # --- Länge angleichen (robust gegen Drift) ---
    n = min(len(candidates), len(mean_np), len(std_np))
    mean_np = mean_np[:n]
    std_np  = std_np[:n]
    candidates_plot = candidates.iloc[:n]

    x = np.arange(n)
    names = candidates_plot["structure_name"].values if "structure_name" in candidates_plot.columns else None

    plt.figure(figsize=(12, 4.5))
    plt.plot(x, mean_np, label="Predicted mean")
    plt.fill_between(x, mean_np - std_np, mean_np + std_np, alpha=0.3, label="±1σ")

    plt.title(f"Iteration {iteration}: GP mean ± 1σ over candidates")
    plt.xlabel("Candidate index")
    plt.ylabel(f"GP prediction (scaled {label})")
    plt.grid(alpha=0.3)
    plt.legend()
    plt.tight_layout()

    # optional: Namen (sparsam)
    if names is not None:
        if n <= 30:
            plt.xticks(x, names, rotation=90)
        else:
            step = max(1, n // 30)
            plt.xticks(x[::step], names[::step], rotation=90)

    if save:
        plt.savefig(f"{fname_prefix}_iter_{iteration:03d}_400.png", dpi=200, bbox_inches="tight")

    plt.show()


In [3]:
def plot_gp_results(candidates, mean_np, std_np, ei_np, pi_np, iteration, label="target_value"):
    # --- Fix: Längen angleichen ---
    n = min(len(candidates), len(mean_np), len(std_np), len(ei_np), len(pi_np))
    candidates = candidates.iloc[:n].copy()
    mean_np = mean_np[:n]
    std_np  = std_np[:n]
    ei_np   = ei_np[:n]
    pi_np   = pi_np[:n]

    x = np.arange(len(candidates))  # Struktur-Index
    names = candidates["structure_name"].values

    fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    plt.subplots_adjust(hspace=0.3)

    axs[0].plot(x, mean_np, color="C0", label="Predicted Mean")
    axs[0].fill_between(x, mean_np - std_np, mean_np + std_np, color="C0", alpha=0.3, label="Uncertainty (±1σ)")
    axs[0].set_ylabel("Predicted Mean")
    axs[0].set_title(f"Iteration {iteration}: GP Prediction (mean ± std)")
    axs[0].legend()
    axs[0].grid(alpha=0.3)

    axs[1].plot(x, ei_np, color="C1")
    axs[1].set_ylabel("Expected Improvement")
    axs[1].set_title("Expected Improvement over Candidates")
    axs[1].grid(alpha=0.3)

    axs[2].plot(x, pi_np, color="C2")
    axs[2].set_ylabel("Probability of Improvement")
    axs[2].set_xlabel("Candidate Structure Index")
    axs[2].set_title("Probability of Improvement")
    axs[2].grid(alpha=0.3)

    if len(names) <= 30:
        axs[2].set_xticks(x)
        axs[2].set_xticklabels(names, rotation=90)
    else:
        step = max(1, len(names)//30)
        axs[2].set_xticks(x[::step])
        axs[2].set_xticklabels(names[::step], rotation=90)

    plt.tight_layout()
    #plt.savefig("BO_BO_BO_BO.png", dpi=300, bbox_inches="tight")
    plt.show()


In [4]:
def is_bin_column(col) -> bool:
    """
    True für:
      - 'bin_0', 'bin_1', ... (beliebige nichtnegative Integer)
      - auch für numerische Spaltennamen wie 0, 1, '0', '1' (optional nützlich)
    """
    # numerische Spaltennamen zulassen (z. B. 0, 1, 2)
    if isinstance(col, (int, np.integer)):
        return True

    s = str(col)
    if s.isdigit():                 # '0', '1', ...
        return True
    if re.fullmatch(r"bin_\d+", s): # 'bin_0', 'bin_1', ...
        return True
    return False

MODELL GP

In [5]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_gp(xt_train, yt_train, training_iterations=100):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(xt_train, yt_train, likelihood)

    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.2)
    mll = ExactMarginalLogLikelihood(likelihood, model)

    losses = []

    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(xt_train)
        loss = -mll(output, yt_train)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

    return model.eval(), likelihood.eval(), losses

DATEN EINLESEN & FILTERN

In [6]:
#dft_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/dft_data_temp_pressure_swingswingswing.csv")
dft_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/dft_fckin_clean_kond_64grid.csv")

dft_data["density_bulk"] = (
    dft_data["density_bulk"]
    .astype(str)                            # sicherstellen, dass alles string ist
    .str.strip()                             # Leerzeichen weg
    .str.replace('[', '', regex=False)       # "[" entfernen
    .str.replace(']', '', regex=False)       # "]" entfernen
)
dft_data["density_bulk"] = pd.to_numeric(dft_data["density_bulk"], errors="coerce")
#expV_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP_hist_no_pressure_no_chem_20b_swing.csv")
expV_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP_hist_no_pressure_no_chem_100b.csv")
data = pd.merge(dft_data, expV_data, 'inner', on=["structure_name", "temperature_kelvin"])
feature_columns = [col for col in data.columns if is_bin_column(col)]
data = data[data.beladung_mol_per_kg > 0]
data = data[(data.temperature_kelvin == 400) & (data.pressure_bar == 100)]

data.head()

,Unnamed: 0.1,Unnamed: 0,structure_name,pressure_bar,temperature_kelvin,volume_kubAng,grid.dv,density_Atmos_per_kubAng,density_bulk,fraction_of_used_points,...,bin_92,bin_93,bin_94,bin_95,bin_96,bin_97,bin_98,bin_99,x_min_y,x_max_y
5580,100750,100750,DDR,100.0,400.0,6715.860313,0.025619,855.049395,0.001884,0.082912,...,143,146,149,142,96,68,109,237983,-15.0,10.1
5581,100751,100751,RRO,100.0,400.0,1007.699890,0.003844,565.118379,0.001884,0.045502,...,160,128,136,176,112,92,144,247236,-15.0,10.1
5582,100752,100752,MER,100.0,400.0,1954.329977,0.007455,727.048515,0.001884,0.072937,...,192,160,320,160,96,96,288,239488,-15.0,10.1
5583,100753,100753,EOS,100.0,400.0,682.685342,0.002604,855.860658,0.001884,0.064011,...,212,148,196,156,128,176,164,242288,-15.0,10.1
5584,100754,100754,CFI,100.0,400.0,1908.333111,0.007280,508.661083,0.001884,0.115356,...,96,160,160,80,80,112,80,229328,-15.0,10.1


FEATURES/LABEL CUSTOM

In [7]:
#data["beladung_pro_vol"] = data["beladung_atoms"] / data["volume_kubAng"]

data["beladung_pro_vol"] = (
    data["beladung_atoms"]
    .div(data["density_bulk"], axis=0)
    .div(data["volume_kubAng"], axis=0)
)

data[feature_columns] = (
    data[feature_columns]
    .multiply(data["grid.dv"], axis=0)
    .div(data["volume_kubAng"], axis=0)
)


NORMALIZE

In [8]:
normalize_feature = True
normalize_labels = True

FOLD - TRAINING - PREDICTION

In [9]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

label = "beladung_pro_vol"
X = data[feature_columns].values 
y = data[label].values 

ids = data.index.values

split_info = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
    x_train = torch.tensor(X[train_idx], dtype=torch.float64)
    y_train = torch.tensor(y[train_idx], dtype=torch.float64)
    x_test = torch.tensor(X[test_idx], dtype=torch.float64)
    y_test = torch.tensor(y[test_idx], dtype=torch.float64)

    train_ids = ids[train_idx]
    test_ids = ids[test_idx]

    test_df = data.iloc[test_idx].copy()
    test_df["fold"] = fold

    if normalize_feature:
        feature_transformer = MinMaxScaler()
        feature_transformer.fit(x_train)
        xt_train = torch.tensor(feature_transformer.transform(x_train), dtype=torch.float64)
        xt_test = torch.tensor(feature_transformer.transform(x_test), dtype=torch.float64) #*2
    else:
        xt_train = x_train
        xt_test = x_test

    # Label-Normalisierung
    if normalize_labels:
        label_transformer = MinMaxScaler()  # oder StandardScaler()
        label_transformer.fit(y_train.unsqueeze(1))
        yt_train = torch.tensor(label_transformer.transform(y_train.unsqueeze(1)).flatten(), dtype=torch.float64)
        yt_test = torch.tensor(label_transformer.transform(y_test.unsqueeze(1)).flatten(), dtype=torch.float64)
    else:
        yt_train = y_train
        yt_test = y_test

    # Training
    model, likelihood, losses = train_gp(xt_train, yt_train, training_iterations=200)

    # Prediction
    with torch.no_grad():
        prediction = model(xt_test)
        inverse_transformed_prediction = label_transformer.inverse_transform(
            prediction.mean.unsqueeze(1)
        ).squeeze()
        inverse_transformed_prediction = np.where(
            inverse_transformed_prediction > 0, inverse_transformed_prediction, 0
        )

    # Ergebnisse
    test_df[f"{label}_pred"] = inverse_transformed_prediction
    test_df["abs_rel_deviation"] = np.abs(
        (test_df[label] - test_df[f"{label}_pred"]) / test_df[label] * 100
    )

    split_info.append(test_df)

results = pd.concat(split_info, ignore_index=True)
r2 = r2_score(results[label], results[f"{label}_pred"])
mae = mean_absolute_error(results[label], results[f"{label}_pred"])
median_ape = results["abs_rel_deviation"].median()

results["R2"] = r2
results["MAE"] = mae
results["Median_APE_percent"] = median_ape
results

,Unnamed: 0.1,Unnamed: 0,structure_name,pressure_bar,temperature_kelvin,volume_kubAng,grid.dv,density_Atmos_per_kubAng,density_bulk,fraction_of_used_points,...,bin_95,bin_96,bin_97,bin_98,bin_99,x_min_y,x_max_y,R2,MAE,Median_APE_percent
0,100754,100754,CFI,100.0,400.0,1908.333111,0.007280,508.661083,0.001884,0.115356,...,0.000305,0.000305,0.000427,0.000305,0.874817,-15.0,10.1,0.544345,0.234173,13.443852
1,100768,100768,AWO,100.0,400.0,2632.814047,0.010043,760.691020,0.001884,0.034180,...,0.000427,0.000183,0.000427,0.000427,0.958679,-15.0,10.1,0.544345,0.234173,13.443852
2,100776,100776,WEI,100.0,400.0,1211.777916,0.004623,598.468207,0.001884,0.045776,...,0.000793,0.000671,0.000732,0.000854,0.939514,-15.0,10.1,0.544345,0.234173,13.443852
3,100794,100794,SBE,100.0,400.0,9320.777439,0.035556,892.040301,0.001884,0.235474,...,0.001221,0.000549,0.000366,0.000671,0.750488,-15.0,10.1,0.544345,0.234173,13.443852
4,100795,100795,AFT,100.0,400.0,4780.480210,0.018236,825.168468,0.001884,0.140762,...,0.000977,0.000809,0.000824,0.000687,0.843811,-15.0,10.1,0.544345,0.234173,13.443852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,100836,100836,SFW,100.0,400.0,7157.664035,0.027304,838.330146,0.001884,0.140015,...,0.000900,0.000755,0.000694,0.000771,0.844582,-15.0,10.1,0.544345,0.234173,13.443852
123,100842,100842,BPH,100.0,400.0,1914.802255,0.007304,836.973388,0.001884,0.185516,...,0.000618,0.000504,0.000664,0.000755,0.799156,-15.0,10.1,0.544345,0.234173,13.443852
124,100852,100852,MTT,100.0,400.0,1318.209551,0.005029,436.978678,0.001884,0.061676,...,0.000397,0.000275,0.000458,0.000458,0.930176,-15.0,10.1,0.544345,0.234173,13.443852
125,100856,100856,BIK,100.0,400.0,642.881753,0.002452,329.916240,0.001884,0.020203,...,0.000427,0.000305,0.000366,0.000244,0.972473,-15.0,10.1,0.544345,0.234173,13.443852


ALLE DATEN

In [10]:
print(f"R²                        : {r2_score(results[label], results[f'{label}_pred']):.4f}")
print(f"MAE                       : {mean_absolute_error(results[label], results[f'{label}_pred']):.4f}")
print(f"Median APE                : {results['abs_rel_deviation'].median():.2f}%")
print(f"Mean APE                  : {results['abs_rel_deviation'].mean():.2f}%")
print(f"Final Loss                : {losses[-1]:.4f}")

count = (results['abs_rel_deviation'] > 20).sum()
print(f"Abs rel dev > 20%         : {count} out of {len(results)}")
print(f"Max abs rel dev           : {results['abs_rel_deviation'].max():.2f}%")


R²                        : 0.5443
MAE                       : 0.2342
Median APE                : 13.44%
Mean APE                  : 7909.04%
Final Loss                : -0.8154
Abs rel dev > 20%         : 36 out of 127
Max abs rel dev           : 878749.05%


EINZELNE FOLDS

In [11]:
for fold, group in results.groupby("fold"):
    print(f"\nFold {fold}")
    print(f"R²           : {r2_score(group[label], group[f'{label}_pred']):.4f}")
    print(f"Median APE   : {group['abs_rel_deviation'].median():.2f}%")
    print(f"Mean APE     : {group['abs_rel_deviation'].mean():.2f}%")
    print(f"Max ARD      : {group['abs_rel_deviation'].max():.2f}%")
    print(f"Final Loss   : {losses[-1]:.4f}")



Fold 1
R²           : -0.6384
Median APE   : 18.86%
Mean APE     : 20.07%
Max ARD      : 66.02%
Final Loss   : -0.8154

Fold 2
R²           : 0.4339
Median APE   : 15.07%
Mean APE     : 18.12%
Max ARD      : 62.74%
Final Loss   : -0.8154

Fold 3
R²           : 0.4046
Median APE   : 19.92%
Mean APE     : 28.38%
Max ARD      : 111.51%
Final Loss   : -0.8154

Fold 4
R²           : 0.7459
Median APE   : 9.97%
Mean APE     : 67835.99%
Max ARD      : 878749.05%
Final Loss   : -0.8154

Fold 5
R²           : 0.8200
Median APE   : 8.81%
Mean APE     : 12.03%
Max ARD      : 49.81%
Final Loss   : -0.8154

Fold 6
R²           : -0.1418
Median APE   : 12.17%
Mean APE     : 15.04%
Max ARD      : 33.61%
Final Loss   : -0.8154

Fold 7
R²           : 0.5907
Median APE   : 13.83%
Mean APE     : 17.84%
Max ARD      : 76.52%
Final Loss   : -0.8154

Fold 8
R²           : 0.8456
Median APE   : 13.76%
Mean APE     : 10062.92%
Max ARD      : 120609.82%
Final Loss   : -0.8154

Fold 9
R²           : 0.1780
Med

PARITY PLOT - ALT

In [12]:
alt.Chart(results).mark_circle(size=60).encode(
    x=label,
    y=f"{label}_pred",
    color="fold:N",
    tooltip=["structure_name", label, f"{label}_pred", "abs_rel_deviation", "fold"]
).interactive()

alt.Chart(...)

In [13]:
"""
alt.Chart(results[results["fold"] == 1]).mark_circle(size=60).encode(
    x=label,
    y=f"{label}_pred",
    color="fold:N",
    tooltip=["structure_name", label, f"{label}_pred", "abs_rel_deviation", "fold"]
).interactive()
"""




'\nalt.Chart(results[results["fold"] == 1]).mark_circle(size=60).encode(\n    x=label,\n    y=f"{label}_pred",\n    color="fold:N",\n    tooltip=["structure_name", label, f"{label}_pred", "abs_rel_deviation", "fold"]\n).interactive()\n'

In [14]:
import numpy as np
import matplotlib.pyplot as plt



DATA ANALYSIS

In [15]:
high_dev_names = results.loc[results["abs_rel_deviation"] > 20, "structure_name"].tolist()
filtered_results = results[results["structure_name"].isin(high_dev_names)]
filtered_results

filter_res = results.sort_values(by="beladung_mol_per_kg", ascending=False)

filter_res.head()

,Unnamed: 0.1,Unnamed: 0,structure_name,pressure_bar,temperature_kelvin,volume_kubAng,grid.dv,density_Atmos_per_kubAng,density_bulk,fraction_of_used_points,...,bin_95,bin_96,bin_97,bin_98,bin_99,x_min_y,x_max_y,R2,MAE,Median_APE_percent
20,100828,100828,AFY,100.0,400.0,1132.678632,0.004321,1372.376814,0.001884,0.182808,...,0.000626,0.000969,0.000809,0.000923,0.799072,-15.0,10.1,0.544345,0.234173,13.443852
23,100848,100848,HZF,100.0,400.0,8831.383692,0.033689,921.759995,0.001884,0.391144,...,0.000610,0.000793,0.000793,0.000732,0.593201,-15.0,10.1,0.544345,0.234173,13.443852
11,100864,100864,CLO,100.0,400.0,17253.512704,0.065817,928.044395,0.001884,0.300110,...,0.000183,0.001282,0.000549,0.000732,0.680664,-15.0,10.1,0.544345,0.234173,13.443852
21,100844,100844,RHO,100.0,400.0,3320.619714,0.012667,1158.280199,0.001884,0.174805,...,0.000916,0.000366,0.000732,0.000000,0.810547,-15.0,10.1,0.544345,0.234173,13.443852
69,100767,100767,SYT,100.0,400.0,10455.274756,0.039884,966.776238,0.001884,0.243774,...,0.000854,0.000488,0.000549,0.000549,0.739319,-15.0,10.1,0.544345,0.234173,13.443852


BAYESIAN OPTIMIZATION

In [16]:
from torch.distributions import Normal

def af_log_expIm(mean, var, best_f, xi=0.01):
    """Logarithmic Expected Improvement acquisition function."""

    std = torch.sqrt(var)
    std_safe = torch.clamp(std, min=1e-9)  # Avoid division by zero
    z = (mean - best_f - xi) / std_safe
    normal = Normal(torch.zeros_like(z), torch.ones_like(z))
    cdf = normal.cdf(z)
    pdf = torch.exp(normal.log_prob(z))

    ei = std * (z * cdf + pdf)

    ei_safe = torch.clamp(ei, min=1e-9)  # Avoid log(0)
    log_ei = torch.log(ei_safe)
    return log_ei
    


In [17]:
candidates = data.copy() # zunächst gefilteret, später alle Daten

patience = 10

n_initial = 1 # Anzahl der initialen Trainingspunkte
initial_indices = candidates.nsmallest(n_initial, label).index # hier geht auch random

print(f"Initial training points:")
for idx in initial_indices:
    print(f"  Index {idx}, Structure {candidates.loc[idx, 'structure_name']}, {label}: {candidates.loc[idx, label]:.4f}")

# Transfer from candidates to selection
selected = candidates.loc[initial_indices]
candidates = candidates.drop(initial_indices)
best = [selected[label].max()]

for i in range(100):
    if len(best) >= patience:
        if len(np.unique(best[-patience:])) == 1:
            print(f"Early stopping at iteration {i} due to no improvement in the last {patience} iterations.")
            break
    
    feature_transoformer = MinMaxScaler()
    label_transformer = MinMaxScaler()

    train_x = torch.tensor(feature_transoformer.fit_transform(selected[feature_columns].values))
    train_y = torch.tensor(label_transformer.fit_transform(selected[[label]].values)).flatten()

    test_x = torch.tensor(feature_transoformer.transform(candidates[feature_columns].values))

    model, likelihood, _ = train_gp(train_x, train_y, 250)
    with torch.no_grad():
        prediction = model(test_x)
        mean, var = prediction.mean, prediction.variance
    
    best_f = train_y.max()

    log_ei = af_log_expIm(mean, var, best_f, 0.01 * best_f)

    # Select the candidate with the highest acquisition value
    index = torch.argmax(log_ei).item()
    best.append(selected[label].max())
    print(f"Iteration: {i}, Current Best: {selected[label].max():.2e}")
    selected = pd.concat([selected, candidates.iloc[[index]]])
    canidates = candidates.drop(candidates.index[index])
    plot_gp_results(candidates, mean_np, std_np, ei_np, pi_np, iteration=i, label=label)
    
print(f"Best Value after {len(best)} iterations: {best[-1]}")

mean_np = mean.detach().cpu().numpy().flatten()
var_np = var.detach().cpu().numpy().flatten()
std_np = np.sqrt(var_np)
ei_np = torch.exp(log_ei).detach().cpu().numpy().flatten()

# --- Probability of Improvement ---
best_f = train_y.max().item()
z = (mean_np - best_f) / std_np
pi_np = norm.cdf(z)


# --- Plot am Ende ---
plot_gp_results(candidates, mean_np, std_np, ei_np, pi_np, iteration=i, label=label)
    

Initial training points:
  Index 5685, Structure LIT, beladung_pro_vol: 0.0000
Iteration: 0, Current Best: 4.16e-05


NameError: name 'mean_np' is not defined

In [ ]:
plt.plot(best)
plt.hlines(data[label].max(), 0, len(best)-1, colors='r', linestyles='dashed')
plt.ylabel("Beladung pro Volumen [Atoms/Å³]")
plt.xlabel("Iterations")

In [ ]:
def log_expected_improvement(mean, var, best_f, xi=0.0):
    """
    Calculate the log of the Expected Improvement acquisition function.
    
    Args:
        mean: Predicted mean from the GP model (torch.Tensor)
        var: Predicted variance from the GP model (torch.Tensor)
        best_f: Best observed function value so far (float or torch.Tensor)
        xi: Exploration-exploitation trade-off parameter (default: 0.01)
    
    Returns:
        log_ei: Log of Expected Improvement for each test point
    """
    std = torch.sqrt(var)
    
    # Avoid division by zero
    std_safe = torch.clamp(std, min=1e-9)
    
    # Standardized improvement
    z = (mean - best_f - xi) / std_safe
    
    # Normal distribution
    normal = Normal(torch.zeros_like(z), torch.ones_like(z))
    
    # Calculate log EI using log-sum-exp trick for numerical stability
    # EI = std * (z * Phi(z) + phi(z))
    # where Phi is CDF and phi is PDF
    
    cdf = normal.cdf(z)
    pdf = torch.exp(normal.log_prob(z))
    
    # For numerical stability, handle cases where std is very small
    ei = std * (z * cdf + pdf)
    
    # Clamp to avoid log(0)
    ei_safe = torch.clamp(ei, min=1e-10)
    log_ei = torch.log(ei_safe)
    
    return log_ei
candidates = data.copy()

# patiences for convergence criteria
patience = 10 # 10 trials without changing the best value to interrupt search

# select initial number of worst (=smalles adsorpted amount?)
n_initial = 1
initial_indices = candidates.nsmallest(n_initial, label).index

print(f"Initial selections:")
for idx in initial_indices:
    print(f"  Index={idx}, Structure={candidates.loc[idx].structure_name}, {label}={candidates.loc[idx][label]:.2e}")

# Transfer entries from candidates to selection
selected = candidates.loc[initial_indices]
candidates = candidates.drop(initial_indices)
best = [selected[label].max()]

for i in range(100):
    if len(best) >= patience:
        if len(np.unique(best[-patience:])) == 1:
            break
        
    feature_transformer = MinMaxScaler()
    label_transformer = MinMaxScaler()

    train_x = torch.tensor(feature_transformer.fit_transform(selected[feature_columns].values))
    train_y = torch.tensor(label_transformer.fit_transform(selected[[label]].values)).flatten()    
    
    test_x = torch.tensor(feature_transformer.transform(candidates[feature_columns].values))

    model, likelihood, _ = train_gp(train_x, train_y, 250)
    with torch.no_grad():
        prediction = model(test_x)
        mean, var = prediction.mean, prediction.variance

    plot_gp_mean_with_uncertainty(candidates, mean, var, iteration=i, label=label, save=True)

    # Get the best observed value (in transformed space)
    best_f = train_y.max()
    
    # Calculate log expected improvement
    log_ei = log_expected_improvement(mean, var, best_f, 0.0)
    
    # Select the next point with highest EI
    index = torch.argmax(log_ei).item()
    best.append(selected[label].max())
    print(f"Iteration: {i}, Current Best: {selected[label].max():.2e}")
    selected = pd.concat([selected, candidates.iloc[[index]]])
    candidates = candidates.drop(candidates.index[index])

print(f"Best value after {len(best)} iterations: {best[-1]}")

#plot_gp_results(candidates, mean_np, std_np, ei_np, pi_np, iteration=20, label=label)
